In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def euclidean_distance(point1, point2):
    distance = 0
    for a, b in zip(point1, point2):
        distance += (b-a)**2
    return distance ** 2 

class KMeans:

    def __init__(self, nb_cluster=5, max_iters=100, plot_steps=False):
        self.nb_cluster = nb_cluster
        self.max_iters = max_iters
        self.plot_steps = plot_steps

        #store the sample indices for each cluster
        self.clusters = [[] for _ in range(self.nb_cluster)]

        #store the centroids(mean vector) for each cluster
        self.centroids = []

    def predict(self, X):
        self.X = X
        #storing number of samples and each features of the dataset
        self.n_samples, self.n_features = X.shape

        #Initialize the centroids
            ## Firstly, we generate randomly the indices between n_samples indices 
        random_sample_indices = np.random.choice(self.n_samples, self.nb_cluster, replace=False)
            ## Secondly, we initialize the centroids
        self.centroids = [self.X[indice] for indice in random_sample_indices]

        #Optimization clusters
        for _ in range(self.max_iters):
            #assign samples to closest centroids (create clusters)
            self.clusters = self._create_clusters(self.centroids)

            if self.plot_steps: 
                self.plot()

            #calculate new centroids from the clusters
            centroids_old = self.centroids
            self.centroids = self._get_centroids(self.clusters)#here we update centroids through this function

            if self._is_converged(centroids_old, self.centroids):
                break
            if self.plot_steps: 
                self.plot()

        #Classify sample as the index of their clusters
        return self._get_cluster_labels(self.clusters)

    def _create_clusters(self, centroids):
        #assign the sample to the closest centroids 
        clusters = [[] for _ in range(self.nb_cluster)]
        for idx, sample in enumerate(self.X):
            ## calculate distance between each sample of dataset and each centroid and we retrieve the closest centroid of each sample
            centroid_idx = self._closest_centroid(sample, centroids) #retrieve the closest centroid indicee 
            clusters[centroid_idx].append(idx) #we append indice of the sample in the corresponding cluster
        return clusters
    
    def _closest_centroid(self, sample, centroids):
        #distance of the currrent sample to each centroid
        distances = [euclidean_distance(sample, centroid) for centroid in centroids]
        closest_idx = np.argmin(distances) #return array index
        return closest_idx

    def _get_centroids(self, clusters):
        #assign the mean value of the cluster to centroids
        centroids = np.zeros(self.nb_cluster, self.n_features)
        for cluster_idx, cluster in enumerate(clusters):
            cluster_mean = np.mean(self.X[cluster], axis=0)# retrieve samples in self.X of a cluster and calculate the mean of each feature
            centroids[cluster_idx] = cluster_mean
        return centroids

    def _is_converged(self, centroids_old, new_centroids):
        #chack the distance between old and new centroids, for all centroids
        distances = [euclidean_distance(centroids_old[i], new_centroids[i]) for i in range(self.nb_clusters)] 
        return sum(distances) == 0

    def _get_cluster_labels(self, clusters):
        #each sample will get he label of the cluster it was assigned to 
        labels = np.empty(self.n_samples)
        # we retrieve from each cluster his indice and itself
        for cluster_idx, cluster in enumerate(clusters):
            # we retrieve each sample indice of the cluster and we store his indice in the label
            for sample_idx in cluster:
                labels[sample_idx] = cluster_idx

        return labels

    def plot(self):
        fig, ax = plt.subplots(figsize=(12, 8))

        for i, index in enumerate(self.clusters):
            point = self.X[index].T
            ax.scatter(*point)
        
        for point in self.centroids:
            ax.scatter(*point, marker="x", color="black", linewidth=2)

        plt.show()


    


In [ ]:
#Testing